In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [12]:
# define the dataframe columns
column_names = ['City', 'State', 'Latitude', 'Longitude'] 
cities_latlong = [['Savannah','Georgia',32.0809263, -81.0911768],['Charleston','South Carolina', 32.7876012, -79.9402728],
                  ['Orlando','Florida',28.5421109, -81.3790304],['New Orleans','Louisiana', 29.9499323, -90.0701156],
                  ['San Francisco','California',37.7790262, -122.4199061],['Boston','Massachusetts', 42.3602534, -71.0582912],
                  ['Sedona','Arizona',34.8679599, -111.7617165],['Key West','Florida', 24.5625566, -81.7724368],
                  ['Chicago','Illinois',41.8755616, -87.6244212],['Houston','Texas', 29.7589382, -95.3676974]
                 ]
# instantiate the dataframe
Destination_Cities = pd.DataFrame(cities_latlong, columns=column_names)

In [13]:
Destination_Cities = Destination_Cities.sort_values(by=['State']).reset_index(drop=True)

In [14]:
Destination_Cities

,City,State,Latitude,Longitude
0,Sedona,Arizona,34.867960,-111.761717
1,San Francisco,California,37.779026,-122.419906
2,Orlando,Florida,28.542111,-81.379030
3,Key West,Florida,24.562557,-81.772437
4,Savannah,Georgia,32.080926,-81.091177
5,Chicago,Illinois,41.875562,-87.624421
6,New Orleans,Louisiana,29.949932,-90.070116
7,Boston,Massachusetts,42.360253,-71.058291
8,Charleston,South Carolina,32.787601,-79.940273
9,Houston,Texas,29.758938,-95.367697


In [15]:
address = 'United States'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the requested City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the requested City are 39.7837304, -100.4458825.


In [18]:
# create map of Destination Cities using latitude and longitude values
map_destinationcities = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for lat, lng, city, state in zip(Destination_Cities['Latitude'], Destination_Cities['Longitude'], Destination_Cities['City'], Destination_Cities['State']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_destinationcities)  
    
map_destinationcities

In [19]:
CLIENT_ID = 'EF44GCH2KBU0HPY4XVHSYL2MY2XO3MFQ1OMZ5IFICZHHIE2F' # your Foursquare ID
CLIENT_SECRET = 'AKGOTNP3DP3B1EXOHKMPJNPI4DCH2XH2H1EFPZOYEGBO3X5U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EF44GCH2KBU0HPY4XVHSYL2MY2XO3MFQ1OMZ5IFICZHHIE2F
CLIENT_SECRET:AKGOTNP3DP3B1EXOHKMPJNPI4DCH2XH2H1EFPZOYEGBO3X5U


In [21]:
Destination_Cities.loc[0, 'City']

'Sedona'

In [22]:
city_latitude = Destination_Cities.loc[0, 'Latitude'] # neighborhood latitude value
city_longitude = Destination_Cities.loc[0, 'Longitude'] # neighborhood longitude value

city_name = Destination_Cities.loc[0, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Sedona are 34.8679599, -111.7617165.


In [23]:
# type your answer here

#The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude,
    city_longitude,  
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=EF44GCH2KBU0HPY4XVHSYL2MY2XO3MFQ1OMZ5IFICZHHIE2F&client_secret=AKGOTNP3DP3B1EXOHKMPJNPI4DCH2XH2H1EFPZOYEGBO3X5U&v=20180605&ll=34.8679599,-111.7617165&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edaef977828ae001b1d7b1f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Sedona',
  'headerFullLocation': 'Sedona',
  'headerLocationGranularity': 'city',
  'totalResults': 63,
  'suggestedBounds': {'ne': {'lat': 34.872459904500005,
    'lng': -111.75624208509173},
   'sw': {'lat': 34.8634598955, 'lng': -111.76719091490828}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c5599e1479fc928bbb32395',
       'name': 'Sedona Red Rocks',
       'location': {'lat': 34.8669513938354,
        'lng': -111.76270333452997,
        'labeledLatLngs': [{'label': 'display',
          'lat': 34.8669513938354,
        

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Sedona Red Rocks,Scenic Lookout,34.866951,-111.762703
1,Cress on Oak Creek,French Restaurant,34.868218,-111.759917
2,Pink Jeep Tours - Sedona,Tourist Information Center,34.869200,-111.760958
3,Wildflower Bread Company,Restaurant,34.867936,-111.762117
4,"Hyatt Residence Club Sedona, Piñon Pointe",Hotel,34.867478,-111.763548


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

63 venues were returned by Foursquare.


In [28]:
def getCityVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    city_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    city_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(city_venues)

In [29]:
DestinationCity_venues = getCityVenues(names=Destination_Cities['City'],
                                   latitudes=Destination_Cities['Latitude'],
                                   longitudes=Destination_Cities['Longitude']
                                  )

Sedona
San Francisco
Orlando
Key West
Savannah
Chicago
New Orleans
Boston
Charleston
Houston


In [30]:
print(DestinationCity_venues.shape)
DestinationCity_venues.head()

(753, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sedona,34.86796,-111.761717,Sedona Red Rocks,34.866951,-111.762703,Scenic Lookout
1,Sedona,34.86796,-111.761717,Cress on Oak Creek,34.868218,-111.759917,French Restaurant
2,Sedona,34.86796,-111.761717,Pink Jeep Tours - Sedona,34.869200,-111.760958,Tourist Information Center
3,Sedona,34.86796,-111.761717,Wildflower Bread Company,34.867936,-111.762117,Restaurant
4,Sedona,34.86796,-111.761717,"Hyatt Residence Club Sedona, Piñon Pointe",34.867478,-111.763548,Hotel


In [31]:
DestinationCity_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Boston,98,98,98,98,98,98
Charleston,84,84,84,84,84,84
Chicago,100,100,100,100,100,100
Houston,43,43,43,43,43,43
Key West,26,26,26,26,26,26
New Orleans,100,100,100,100,100,100
Orlando,49,49,49,49,49,49
San Francisco,97,97,97,97,97,97
Savannah,93,93,93,93,93,93


In [32]:
print('There are {} uniques categories.'.format(len(DestinationCity_venues['Venue Category'].unique())))

There are 175 uniques categories.


In [34]:
# one hot encoding
DestinationCity_onehot = pd.get_dummies(DestinationCity_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DestinationCity_onehot['City'] = DestinationCity_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [DestinationCity_onehot.columns[-1]] + list(DestinationCity_onehot.columns[:-1])
DestinationCity_onehot = DestinationCity_onehot[fixed_columns]

DestinationCity_onehot.head()

,City,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Belgian Restaurant,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Library,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Museum,Music School,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Optical Shop,Organic Grocery,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tiki Bar,Tour Provider,Tourist Information Center,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
DestinationCity_onehot.shape

(753, 176)

In [36]:
DestinationCity_grouped = DestinationCity_onehot.groupby('City').mean().reset_index()
DestinationCity_grouped

,City,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Belgian Restaurant,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Library,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Museum,Music School,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Optical Shop,Organic Grocery,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tiki Bar,Tour Provider,Tourist Information Center,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Boston,0.000000,0.000000,0.051020,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.051020,0.010204,0.000000,0.010204,0.000000,0.010204,0.00,0.010204,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020408,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.010204,0.000000,0.000000,0.040816,0.00000,0.000000,0.000000,0.00,0.010204,0.000000,0.010204,0.010204,0.000000,0.000000,0.00,0.000000,0.020408,0.000000,0.000000,0.000000,0.030612,0.000000,0.000000,0.00,0.010204,0.010204,0.000000,0.010204,0.000000,0.000000,0.010204,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.010204,0.020408,0.010204,0.000000,0.010204,0.040816,0.000000,0.00,0.00,0.040816,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.122449,0.020408,0.000000,0.000000,0.010204,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.020408,0.000000,0.00,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.010204,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.030612,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.020408,0.00,0.061224,0.000000,0.020408,0.010204,0.000000,0.010204,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020408,0.000000,0.010204,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.020408
1,Charleston,0.011905,0.000000,0.059524,0

In [54]:
DestinationCity_grouped.shape

(10, 176)

In [55]:
num_top_venues = 5

for hood in DestinationCity_grouped['City']:
    print("----"+hood+"----")
    temp = DestinationCity_grouped[DestinationCity_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Boston----
                 venue  freq
0   Italian Restaurant  0.12
1       Sandwich Place  0.06
2               Bakery  0.05
3  American Restaurant  0.05
4        Historic Site  0.04


----Charleston----
                 venue  freq
0  American Restaurant  0.06
1         Cocktail Bar  0.06
2               Bakery  0.05
3       Sandwich Place  0.04
4                  Bar  0.04


----Chicago----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.04
2           Hotel  0.04
3     Pizza Place  0.04
4          Museum  0.03


----Houston----
                   venue  freq
0                  Hotel  0.12
1           Burger Joint  0.09
2     Italian Restaurant  0.07
3  Performing Arts Venue  0.07
4                   Park  0.07


----Key West----
                 venue  freq
0        Grocery Store  0.12
1  Rental Car Location  0.08
2    Mobile Phone Shop  0.08
3          Pizza Place  0.08
4             Pharmacy  0.08


----New Orleans----
                       venue  freq


In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
DestinationCity_venues_sorted = pd.DataFrame(columns=columns)
DestinationCity_venues_sorted['City'] = DestinationCity_grouped['City']

for ind in np.arange(DestinationCity_grouped.shape[0]):
    DestinationCity_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DestinationCity_grouped.iloc[ind, :], num_top_venues)

DestinationCity_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Boston,Italian Restaurant,Sandwich Place,American Restaurant,Bakery,Historic Site,Hotel,Coffee Shop,Donut Shop,Pub,Deli / Bodega
1,Charleston,American Restaurant,Cocktail Bar,Bakery,Sandwich Place,Bar,Italian Restaurant,Pizza Place,Restaurant,Seafood Restaurant,Shipping Store
2,Chicago,Coffee Shop,Pizza Place,Sandwich Place,Hotel,Boutique,Museum,Pub,Theater,Dance Studio,Music Venue
3,Houston,Hotel,Burger Joint,Italian Restaurant,Park,Performing Arts Venue,Concert Hall,Theater,Food Court,Beer Bar,Sushi Restaurant
4,Key West,Grocery Store,Pizza Place,Rental Car Location,Mobile Phone Shop,Pharmacy,Furniture / Home Store,Deli / Bodega,Resort,Coffee Shop,Clothing Store


In [58]:
# set number of clusters
kclusters = 5

DestinationCity_grouped_clustering = DestinationCity_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DestinationCity_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 4, 3, 0, 1, 2, 1, 2], dtype=int32)

In [59]:
# add clustering labels
DestinationCity_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DestinationCity_merged = Destination_Cities

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DestinationCity_merged = DestinationCity_merged.join(DestinationCity_venues_sorted.set_index('City'), on='City')

DestinationCity_merged # check the last columns!

,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sedona,Arizona,34.867960,-111.761717,2,Hotel,Gift Shop,Coffee Shop,Ice Cream Shop,Tour Provider,New American Restaurant,American Restaurant,Mexican Restaurant,Shopping Mall,Bakery
1,San Francisco,California,37.779026,-122.419906,2,Coffee Shop,Café,Hotel,Vegetarian / Vegan Restaurant,Bakery,Park,Cocktail Bar,Theater,Wine Bar,Gym
2,Orlando,Florida,28.542111,-81.379030,1,Bar,American Restaurant,Coffee Shop,Pub,Mexican Restaurant,Cocktail Bar,Lounge,History Museum,Library,Brewery
3,Key West,Florida,24.562557,-81.772437,3,Grocery Store,Pizza Place,Rental Car Location,Mobile Phone Shop,Pharmacy,Furniture / Home Store,Deli / Bodega,Resort,Coffee Shop,Clothing Store
4,Savannah,Georgia,32.080926,-81.091177,1,Bar,Hotel,Seafood Restaurant,Plaza,Southern / Soul Food Restaurant,Gourmet Shop,Coffee Shop,Park,Candy Store,Museum
5,Chicago,Illinois,41.875562,-87.624421,2,Coffee Shop,Pizza Place,Sandwich Place,Hotel,Boutique,Museum,Pub,Theater,Dance Studio,Music Venue
6,New Orleans,Louisiana,29.949932,-90.070116,0,Hotel,Cajun / Creole Restaurant,Hotel Bar,Café,Italian Restaurant,Bar,Restaurant,French Restaurant,Vietnamese Restaurant,Breakfast Spot
7,Boston,Massachusetts,42.360253,-71.058291,2,Italian Restaurant,Sandwich Place,American Restaurant,Bakery,Historic Site,Hotel,Coffee Shop,Donut Shop,Pub,Deli / Bodega
8,Charleston,South Carolina,32.787601,-79.940273,2,American Restaurant,Cocktail Bar,Bakery,Sandwich Place,Bar,Italian Restaurant,Pizza Place,Restaurant,Seafood Restaurant,Shipping Store
9,Houston,Texas,29.758938,-95.367697,4,Hotel,Burger Joint,Italian Restaurant,Park,Performing Arts Venue,Concert Hall,Theater,Food Court,Beer Bar,Sushi Restaurant


In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DestinationCity_merged['Latitude'], DestinationCity_merged['Longitude'], DestinationCity_merged['City'], DestinationCity_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 0, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Louisiana,Hotel,Cajun / Creole Restaurant,Hotel Bar,Café,Italian Restaurant,Bar,Restaurant,French Restaurant,Vietnamese Restaurant,Breakfast Spot


In [63]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 1, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Florida,Bar,American Restaurant,Coffee Shop,Pub,Mexican Restaurant,Cocktail Bar,Lounge,History Museum,Library,Brewery
4,Georgia,Bar,Hotel,Seafood Restaurant,Plaza,Southern / Soul Food Restaurant,Gourmet Shop,Coffee Shop,Park,Candy Store,Museum


In [64]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 2, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arizona,Hotel,Gift Shop,Coffee Shop,Ice Cream Shop,Tour Provider,New American Restaurant,American Restaurant,Mexican Restaurant,Shopping Mall,Bakery
1,California,Coffee Shop,Café,Hotel,Vegetarian / Vegan Restaurant,Bakery,Park,Cocktail Bar,Theater,Wine Bar,Gym
5,Illinois,Coffee Shop,Pizza Place,Sandwich Place,Hotel,Boutique,Museum,Pub,Theater,Dance Studio,Music Venue
7,Massachusetts,Italian Restaurant,Sandwich Place,American Restaurant,Bakery,Historic Site,Hotel,Coffee Shop,Donut Shop,Pub,Deli / Bodega
8,South Carolina,American Restaurant,Cocktail Bar,Bakery,Sandwich Place,Bar,Italian Restaurant,Pizza Place,Restaurant,Seafood Restaurant,Shipping Store


In [65]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 3, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Florida,Grocery Store,Pizza Place,Rental Car Location,Mobile Phone Shop,Pharmacy,Furniture / Home Store,Deli / Bodega,Resort,Coffee Shop,Clothing Store


In [66]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 4, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Texas,Hotel,Burger Joint,Italian Restaurant,Park,Performing Arts Venue,Concert Hall,Theater,Food Court,Beer Bar,Sushi Restaurant
